# New Device API Prototype

In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.devices.experimental import TestDevicePythonSim

Note: this only works with the new return types workflow:

In [2]:
qml.enable_return()

In [3]:
dev = TestDevicePythonSim()

Add some attributes and properties to match existing required interface.

We will have to adjust the existing workflow to accomodate these changes.

In [4]:
# map the execute function
dev.batch_execute = dev.execute

# map the preprocessing steps
dev.batch_transform = dev.preprocess
dev.expand_fn = lambda circuit, max_expansion: circuit

# give dummy shots. We will be moving these out of the class
dev.shots = None
dev._shot_vector = []
dev.shot_vector = None

# short name needed for validation in one place
dev.short_name = "testpython"

The limited number of these demonstrates the disconnect between the number of methods in the base class and the number used for a typical workflow

## Examples of the prototype working

### Let's try a device gradient!

In [5]:
@qml.qnode(dev, diff_method="device")
def circuit(a):
    ops =[qml.RX(a[0], wires=0),
    qml.CNOT(wires=(0,1)),
    qml.RY(a[1], wires=1),
    qml.RZ(a[2], wires=1)]
    return qml.expval(qml.PauliX(1))

x = qml.numpy.array([1.2, 2.3, 3.4])

In [6]:
circuit(x)

[-0.26124053720169715]

In [7]:
qml.grad(circuit)(x)

array([0.67195027, 0.23341436, 0.06905029])

### Parameter Shift?

In [8]:
@qml.qnode(dev, diff_method=qml.gradients.param_shift)
def circuit(a):
    ops =[qml.RX(a[0], wires=0),
    qml.CNOT(wires=(0,1)),
    qml.RY(a[1], wires=1),
    qml.RZ(a[2], wires=1)]
    return qml.expval(qml.PauliX(1))

x = qml.numpy.array([1.2, 2.3, 3.4])

In [9]:
qml.grad(circuit)(x)

array([0.67195027, 0.23341436, 0.06905029])

### How about backprop with Jax jit?

In [10]:
import jax
from jax import numpy as jnp

In [11]:
x = jnp.array([1.2, 2.3, 3.4])

@jax.jit
@qml.qnode(dev, interface="jax", diff_method="backprop")
def circuit(a):
    ops =[qml.RX(a[0], wires=0),
    qml.CNOT(wires=(0,1)),
    qml.RY(a[1], wires=1),
    qml.RZ(a[2], wires=1)]
    return qml.expval(qml.PauliX(1))

In [12]:
jax.jacobian(circuit)(x)

/Users/christina/Prog/pennylane/pennylane/devices/experimental/custom_device_3_numpydev/jax_simulator.py:51: UserWarning: Explicitly requested dtype <class 'jax.numpy.complex128'> requested in zeros is not available, and will be truncated to dtype complex64. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  state = np.zeros(2**num_indices, dtype=dtype)


[DeviceArray([0.6719504 , 0.23341438, 0.06905031], dtype=float32)]

No substitution of device at QNode level!

Device just dispatches to a different simulator.

In [13]:
circuit.device is dev

True

In [14]:
@qml.qnode(dev, diff_method=None)
def circuit_mutual(x):
    qml.IsingXX(x, wires=[0, 1])
    return qml.mutual_info(wires0=[0], wires1=[1])

circuit_mutual(np.pi/2)

[1.3862943611198906]

## Device Tracking

In [15]:
@qml.qnode(dev, diff_method="device")
def circuit(a):
    ops =[qml.RX(a[0], wires=0),
    qml.CNOT(wires=(0,1)),
    qml.RY(a[1], wires=1),
    qml.RZ(a[2], wires=1)]
    return qml.expval(qml.PauliX(1))

x = qml.numpy.array([1.2, 2.3, 3.4])

In [16]:
def callback(totals=None, history=None, latest=None):
    print("Totals: ", totals)

with qml.Tracker(dev, callback=callback) as tracker:
    circuit(x)
    qml.grad(circuit)(x)

Totals:  {'batches': 1, 'batch_len': 1}
Totals:  {'batches': 1, 'batch_len': 1, 'executions': 1, 'results': -0.26124053720169715}
Totals:  {'batches': 1, 'batch_len': 1, 'executions': 1, 'results': -0.26124053720169715, 'gradients': 1}
Totals:  {'batches': 2, 'batch_len': 2, 'executions': 1, 'results': -0.26124053720169715, 'gradients': 1}
Totals:  {'batches': 2, 'batch_len': 2, 'executions': 2, 'results': -0.5224810744033943, 'gradients': 1}
Totals:  {'batches': 2, 'batch_len': 2, 'executions': 2, 'results': -0.5224810744033943, 'gradients': 2}


## Some nice things we can now do with simulators

### More complicated measurement processes?

Simulator can just use the `StateMeasurement.process_state` method.

In [17]:
@qml.qnode(dev, diff_method=None)
def circuit(x):
    qml.IsingXX(x, wires=(0,1))
    return qml.mutual_info(wires0=[0], wires1=[1])

circuit(1.2)

[1.2519553154145866]

### Native execution of non-commuting observables?

Easily handled at the simulator level.

Diagonalizing gates are handled when taking a measurement, not when executing the circuit

In [18]:
@qml.qnode(dev, diff_method=None)
def circuit(a):
    qml.RX(a, 0)
    return qml.expval(qml.PauliX(0)), qml.expval(qml.PauliZ(0))

with qml.Tracker(dev) as tracker:
    print("Execution: ", circuit(1.2))
    
tracker.totals['executions']

Execution:  [(0.0, 0.36235775447667357)]


1

## Arbitrary wire labels?

In [19]:
@qml.qnode(dev)
def circuit(a):
    qml.RX(a, "a")
    return qml.expval(qml.PauliZ("a"))

circuit(1.2)

[0.36235775447667357]

Preprocessing can map wires to adjacent integers starting from zero. Then simulators can just
treat wire labels as indices!

Makes the simulators job eaiser. This mapping could also occur in "internal preprocessing"

In [20]:
qs = qml.tape.QuantumScript([qml.PauliX("a"), qml.PauliY(10)])
qbatch, post_processing_fn = dev.preprocess(qs)
qbatch[0].circuit

[PauliX(wires=[0]), PauliY(wires=[1])]

# Preprocessing of Script

In [21]:
@qml.qnode(dev, diff_method=None)
def circuit(params):
    qml.StronglyEntanglingLayers(params, wires=(0,1,2,3))
    return qml.expval(qml.PauliZ(3))

In [22]:
n_layers = 4
shape = qml.StronglyEntanglingLayers.shape(n_layers=n_layers, n_wires=4)

rng = np.random.default_rng(seed=42)
params = rng.random(shape)

In [23]:
circuit(params)

[0.246704388316073]

Preprocessing expands till it reaches supported operations

In [24]:
batched_qs, post_process_fn = dev.preprocess(circuit.tape)

print(qml.drawer.tape_text(batched_qs[0]))

0: ──Rot─╭●───────╭X──Rot─╭●────╭X──Rot──────╭●─╭X──Rot──────╭●─────────╭X─┤     
1: ──Rot─╰X─╭●────│───Rot─│──╭●─│──╭X────Rot─│──╰●─╭X────Rot─╰X───╭●────│──┤     
2: ──Rot────╰X─╭●─│───Rot─╰X─│──╰●─│─────Rot─│─────╰●───╭X────Rot─╰X─╭●─│──┤     
3: ──Rot───────╰X─╰●──Rot────╰X────╰●────Rot─╰X─────────╰●────Rot────╰X─╰●─┤  <Z>


## Unsupportable Quantum Script?

In [25]:
%xmode Minimal

@qml.qnode(dev, diff_method=None)
def circuit(theta, phi):
    qml.Beamsplitter(theta, phi, wires=(0,1))
    return qml.expval(qml.PauliX(0))

circuit(1.2, 2.3)


Exception reporting mode: Minimal


NotImplementedError: Beamsplitter(1.2, 2.3, wires=[0, 1]) not supported on device

Allows additional forms of validation:

In [26]:
%xmode Minimal

@qml.qnode(dev)
def circuit():
    [qml.PauliX(i) for i in range(50)]
    return qml.expval(qml.PauliX(0))

circuit()

Exception reporting mode: Minimal


NotImplementedError: Requested execution with 50 qubits. We support at most 30.

# What does this look like internally?

### Separation of driver from interface

Device is just the interface.  Implementation details, like simulators or hardware drivers, can be handled in an additional level of abstraction:

In [27]:
from pennylane.devices.experimental import PlainNumpySimulator, JaxSimulator

In [28]:
jax_sim = JaxSimulator()

In [29]:
[obj for obj in dir(jax_sim) if obj[0] != "_"]

['apply_matrix',
 'apply_matrix_einsum',
 'apply_matrix_tensordot',
 'apply_operation',
 'create_state_vector_state',
 'create_zeroes_state',
 'execute',
 'measure']

Improves the documentation and ease of developement for the simulator.

In [30]:
state = jax_sim.create_zeroes_state(1)
print(state)
state = jax_sim.apply_operation(state, qml.PauliX(0))
print(state)
output = jax_sim.measure(state, qml.expval(qml.PauliZ(0)))
print(output)

[1.+0.j 0.+0.j]
[0.+0.j 1.+0.j]
-1.0


## Required device Interface

In [31]:
fresh_dev = TestDevicePythonSim()

[obj for obj in dir(fresh_dev) if obj[0] != "_"]

['capabilities',
 'execute',
 'execute_and_gradients',
 'gradient',
 'preprocess',
 'register_execute',
 'register_fn',
 'register_gradient',
 'registrations',
 'tracker',
 'vjp']

# Takeaways

The abstract base class will only define a minimal interface required to interact with PennyLane Core.

ABC will not define any implementation.

We can separate out "device" from "single device controller".  Need to define names and definitions for these things.

Need tight specification for return shapes. Both make it easy to get things into that right shape, and easy to validate and correct when the return shape is incorrect.



## Execution Config

Getting the workflow to support this will be work.

In [ ]:
from pennylane.runtime import ExecutionConfig

In [ ]:
config = ExecutionConfig(shots=100, interface="jax")
config

In [ ]:
dev.execute(qs, config)

## Registrations

In [32]:
fresh_dev.registrations

{<FnType.GRADIENT: 4>: {1: <function pennylane.devices.experimental.custom_device_3_numpydev.python_device.gradient(self, qscript: pennylane.tape.qscript.QuantumScript, order: int = 1)>}}

In [33]:
@TestDevicePythonSim.register_gradient(order=2)
def hessian(self, qscript, order: int = 2):
    print("look! I'm computing the hessian!")

fresh_dev2 = TestDevicePythonSim()
qs = qml.tape.QuantumScript([qml.RX(1.2, wires=0)], [qml.expval(qml.PauliZ(0))])

fresh_dev2.gradient(qs, order=2)
fresh_dev2.gradient(qs, order=3)

look! I'm computing the hessian!


ValueError: Device does not support 3 order derivatives

In [ ]:
fresh_dev2.registrations[qml.devices.experimental.FnType.GRADIENT]